In [1]:
%run modules.ipynb
# Change this cell to change the kernel function throughout
kernel_params = {"kernel": "rbf", "gamma": 1}
kernel_func = partial(rbf_kernel, gamma=1)
kernel_type = "gaussian"
import multiprocessing
from multiprocessing import Pool
from time import sleep

In [2]:
"""The file has been created in zip format, reading it"""
zf = zipfile.ZipFile('./MP_dataset.zip') 
df_csv = pd.read_csv(zf.open('MP_dataset.csv'), usecols=['e_hull','gap pbe', 'mu_b','structure', 'elastic anisotropy','bulk modulus', 'shear modulus', 'e_form'])

formE=df_csv['e_form']

"""Counting number of atoms from string"""   

def natoms(string):
    for item in string:
        if string[(string.find('Sites'))+8] == ')':
            n=(string.find('Sites'))+7
            return int(string[n])
        else:
            n1=(string.find('Sites'))+7
            n2=(string.find('Sites'))+8
            return int(string[n1]+string[n2])
        
natoms_list=[]
for i in range(len(df_csv)):
    natoms_list.append(natoms(df_csv['structure'][i]))
    
print(len(formE), len(natoms_list))

83989 83989


In [3]:
select=1000
ase_fra=random.sample(ase.io.read('MP.xyz', index=':'), select)
len(ase_fra)

1000

In [4]:
frames_ase=[]

select1=100
for frame in ase_fra:
    frame.wrap(eps=1e-12)
    frames_ase.append(frame)

frames_ase_random=random.sample(ase_fra, select1)
n_FPS=50
soap = SOAP(soap_type='PowerSpectrum',
           interaction_cutoff=3.5,
           max_radial=5,
           max_angular=5,
           gaussian_sigma_type='Constant',
           gaussian_sigma_constant=0.4,
           cutoff_smooth_width=0.5, 
           normalize=False)
soap_rep= soap.transform(frames_ase_random)
soap_rep

In [5]:
%whos 

Xsparse = soap_rep.get_features(soap)
col_idxs = FPS(n_features_to_select=n_FPS).fit(Xsparse).selected_idx_
Xsparse = Xsparse[:, col_idxs]
Xsparse.shape, max(col_idxs)

Variable                 Type                          Data/Info
----------------------------------------------------------------
Abinit                   ABCMeta                       <class 'ase.calculators.abinit.Abinit'>
ArrayData                AbstractNodeMeta              <class 'aiida.orm.nodes.d<...>a.array.array.ArrayData'>
Atoms                    type                          <class 'ase.atoms.Atoms'>
Axes3D                   type                          <class 'mpl_toolkits.mplot3d.axes3d.Axes3D'>
Calculator               ABCMeta                       <class 'ase.calculators.calculator.Calculator'>
Counter                  type                          <class 'collections.Counter'>
FPS                      ABCMeta                       <class 'skcosmo.feature_s<...>election.simple_fps.FPS'>
IncrementalPCA           ABCMeta                       <class 'sklearn.decomposi<...>ntal_pca.IncrementalPCA'>
KPCovR                   ABCMeta                       <class 'skcosmo.de

((3260, 50), 46608)

In [6]:
Xsparse_tot=[]

step=100
for i in range(0, len(frames_ase), step):
    soap_rep= soap.transform(frames_ase[i:i+step])
    Xsp = soap_rep.get_features(soap)
#     print(Xsp.shape)
    z=[ [0] * (max(col_idxs)-len(Xsp[0])+1) for _ in range(len(Xsp))]
    z=np.asarray(z)
    Xsp=np.append(Xsp, z, axis=1)
#     print(Xsp.shape)
    Xsp = Xsp[:, col_idxs]
    print(Xsp.shape)
    Xsparse_tot.append(Xsp)
    
Xrest=np.concatenate(Xsparse_tot)
Xrest.shape

(2630, 50)
(2768, 50)
(2927, 50)
(2716, 50)
(2795, 50)
(2936, 50)
(3100, 50)
(2681, 50)
(3147, 50)
(3074, 50)


(28774, 50)

In [7]:
l=np.concatenate(ase_fra)

Y=np.zeros((len(l), 1))

natoms_list_magic=[]
start=0
for frame in ase_fra:
    natoms=(len(frame))
    
    if natoms%4==0:
        Y[start:int(start+natoms)] = int(2)
        n=2
    else:
        Y[start:int(start+natoms)] = int(1)
        n=1
    natoms_list_magic.append(n)
    start=start+natoms
print("Y shape is:"+ str(Y.shape)), len(natoms_list_magic)

Y shape is:(28774, 1)


(None, 1000)

In [8]:
# Splits in train and test sets
n_train = int(len(Y)/2)  
n_test = len(Y)-n_train
r_train = np.asarray(range(len(Y)))

np.random.shuffle(r_train)

i_test = list(sorted(r_train[n_train:]))
i_train = list(sorted(r_train[:n_train]))

X_train = Xrest[i_train]
Y_train = Y[i_train]
X_test = Xrest[i_test]
Y_test = Y[i_test]

print(f'Shape of training data is: {X_train.shape}, Shape of testing data is: {X_test.shape}, ||X|| = {np.linalg.norm(X_train)}.') 
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

if X_test.shape!=X_train.shape:
    X_test=X_test[:-1,:]
    Y_test=Y_test[:-1,:]
    
if X_test.shape!=X_train.shape and X_test.shape<X_train.shape:
    X_train=X_train[:-1,:]
    Y_train=Y_train[:-1,:]  

    
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

Shape of training data is: (14387, 50), Shape of testing data is: (14387, 50), ||X|| = 0.05178064324101465.
(14387, 50) (14387, 50) (14387, 1) (14387, 1)
(14387, 50) (14387, 50) (14387, 1) (14387, 1)


In [10]:
x_scaler = StandardFlexibleScaler(column_wise=False).fit(X_train)
y_scaler = StandardFlexibleScaler(column_wise=True).fit(Y_train)
X=Xrest
# Center total dataset
X = x_scaler.transform(X)
Y = y_scaler.transform(Y)

# Center training data
X_train = x_scaler.transform(X_train)
Y_train = y_scaler.transform(Y_train)

# Center training data
X_test = x_scaler.transform(X_test)
Y_test = y_scaler.transform(Y_test)

In [ ]:
kp = KPCovR(mixing=0.8, n_components=2, **kernel_params, center=True, alpha=1E-4)
kp.fit(X_train, Y_train)
print(kp.alpha)
t = kp.transform(X_test)
y = kp.predict(X_test)
fig, ax = plt.subplots(1, 2, figsize=dbl_fig)

plot_projection(Y_test, t, title="KPCovR", fig=fig, ax=ax[0], **cmaps)
plot_regression(Y_test[:, 0], y[:, 0], title="KPCovR", fig=fig, ax=ax[1], **cmaps)

In [ ]:
K_train = gaussian_kernel(X_train, X_train)
K_test = gaussian_kernel(X_test, X_train)

k_scaler = KernelNormalizer().fit(K_train)

K_train = k_scaler.transform(K_train)
K_test = k_scaler.transform(K_test)

In [ ]:
regularization = 1e-3

krr = KernelRidge(alpha=regularization, **kernel_params)

krr.fit(K_train, Y_train)

Y_krr_train = krr.predict(K_train)
Y_krr_test = krr.predict(K_test)

table_from_dict([get_stats(x=X_train, yp=Y_krr_train, y=Y_train, k=K_train), 
                 get_stats(x=X_test,  yp=Y_krr_test, y=Y_test, k=K_test)], 
                 headers = ["Training", "Testing"], 
                 title="Kernel Ridge Regression")

In [ ]:
"""How good is the model? 1 is best"""

krr.score(K_test, Y_test)

In [ ]:
type_kernel=['linear', 'poly', 'rbf']

for i in range(len(type_kernel)):
    
    poly_kernel_svm_clf = Pipeline((
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel=str(type_kernel[i]), degree=5, coef0=0.9, C=5))
            ))

    print(str(type_kernel[i])+': '+ str(poly_kernel_svm_clf.fit(t, Y_test.ravel()). score(t,Y_test.ravel())))


In [ ]:
train_sizes, train_scores, valid_scores = learning_curve(
                                    SVC(kernel='linear'), Xrest, Y, train_sizes=[50, 80, 110], cv=5)